In [1]:
from random import randint

import numpy as np
import torch
import json
from collections import Counter
from tqdm import tqdm
import nltk
import regex as re
import string
from nltk.corpus import stopwords
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression

# Get data - pre-processing and general functions

In [28]:
def word_tokenize(tokens):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.word_tokenize(tokens)]


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc((s))))




def get_2d_spans(text, tokenss):
    spanss = []
    cur_idx = 0
    for tokens in tokenss:
        spans = []
        for token in tokens:
            if text.find(token, cur_idx) < 0:
                print(tokens)
                print("{} {} {}".format(token, cur_idx, text))
                raise Exception()
            cur_idx = text.find(token, cur_idx)
            spans.append((cur_idx, cur_idx + len(token)))
            cur_idx += len(token)
        spanss.append(spans)
    return spanss


def get_word_span(context, wordss, start, stop):
    spanss = get_2d_spans(context, wordss)
    idxs = []
    for sent_idx, spans in enumerate(spanss):
        for word_idx, span in enumerate(spans):
            if not (stop <= span[0] or start >= span[1]):
                idxs.append((sent_idx, word_idx))
    #print(spanss)
    #print(start,stop)
    #print(context[start:stop])
    assert len(idxs) > 0, "{} {} {} {}".format(context, spanss, start, stop)
    return idxs[0], (idxs[-1][0], idxs[-1][1] + 1)
def get_word_idx(context, wordss, idx):
    spanss = get_2d_spans(context, wordss)
    return spanss[idx[0]][idx[1]][0]



def process_tokens1(tokens):
   
    
    tokens = [w for w in tokens if w not in set(string.punctuation)]
   # stop_words = set(stopwords.words('english'))
    #tokens = [w for w in tokens if not w in stop_words]
    
    return tokens



def process_tokens(temp_tokens):
    tokens = []
    
    for token in temp_tokens:
        flag = False
        l = ("-", "\u2212", "\u2014", "\u2013", "/", "~", '"', "'", "\u201C", "\u2019", "\u201D", "\u2018", "\u00B0")
        # \u2013 is en-dash. Used for number to nubmer
        # l = ("-", "\u2212", "\u2014", "\u2013")
        # l = ("\u2013",)
        tokens.extend(re.split("([{}])".format("".join(l)), token))
    
    return tokens

def generatedataset(sentence_list,question_list):
    s_list=[]
    q_list=[]
    a_list=[]
    q_id_list=[]
    
    
    res=[]
    #i=0
    for q in question_list:
        #i+=1
        if(len(q)==0):
            #print(i)
            continue
        ai=q[0][0]
      
        pi=q[0][1]
        si=q[0][2]
        
        res+=[[q[0][3],sentence_list[ai][pi][si],q[0][4], q[0][5]]]
        s_list+=[' '.join(sentence_list[ai][pi][si])]
        q_list+=[' '.join(q[0][3])]
        a_list+=[q[0][4]]
        q_id_list+=[q[0][5]]
        #for i in range(len(sentence_list[ai][pi])):
            #if(i==si):
                #res+=[[q[0][3],sentence_list[ai][pi][i],q[0][4], q[0][5]]]
            #else:
            #    res+=[[q[0][3],sentence_list[ai][pi][i],0, q[0][5]]]
            
            
    return res, s_list,q_list,a_list, q_id_list
def create_index_dict(embed,word_counter):
    word2idx={}
    idx2word={}
    i=1
    word2idx['<UNK>']=0
    idx2word[0]='<UNK>'
    vocab=np.array(['<UNK>'])
    for (key,value) in (embed.items()):
        word2idx[key]=i
        idx2word[i]=key
   #     vocab=np.append(vocab,[key])
        i+=1
    
    
    for key,value in (word_counter.items()):
        key=key.lower()
    
        try:
            embed[key]
           
        except: 
            try:
                word2idx[key]
            except:
                word2idx[key]=i
                idx2word[i]=key
        #        vocab=np.append(vocab,[key])
                i+=1
   
   # assert len(word2idx.keys())==len(vocab)
    return word2idx,idx2word,set(word2idx.keys())

def process(data):
    q_trn=[]
    s_trn=[]
    y_trn=[]
    q_id=[]
    for i in range(len(data)):
       # tokens = [w for w in tokens if w not in set(string.punctuation) ]
        
        
        question=list(filter(None, data[i][0]))#filter empty strin
        sentence=list(filter(None, data[i][1]))#filter empty strin
        index=data[i][2]
       # list1+=[[question,sentence,index]]
        
        q_trn+=[question]
        s_trn+=[sentence]
        y_trn+=[index]
        q_id+=[data[i][3]]
    return q_trn,s_trn,y_trn, q_id

In [3]:
a=[1,2,3]
a.append([4,5,6])
a

[1, 2, 3, [4, 5, 6]]

In [4]:
data_type='training'
source_path = "{}.json".format(data_type)
source_data = json.load(open(source_path, 'r'))

data_type='training'
split=True
nltk.data.path.append("/afs/cs.stanford.edu/u/tianzhao/tz/nltk_data")
sent_tokenize = nltk.sent_tokenize

In [5]:
nltk.data.path.append("/afs/cs.stanford.edu/u/tianzhao/tz/nltk_data")
sent_tokenize = nltk.sent_tokenize
word_counter = Counter()


In [6]:
context_list=[]
sentence_list=[]
question_list=[]
answer_list=[]
answer_index=[]
label_index=[]
label=[]
idxs=[]


word_counter = Counter()

for ai, article in enumerate(tqdm(source_data['data'])):
    s=[]
    p = []
    for pi, para in enumerate(article['paragraphs']):
        # wordss
        context = para['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
       
        si = list(map(word_tokenize, sent_tokenize(context)))
       # print(si)
        si = [process_tokens(tokens) for tokens in si]  # process tokens
        for sentence in si:
            for word in sentence:
                 word_counter[word] += 1
 
        # given xi, add chars
        s.append(si)
        p.append(context)

        indexi = [ai, pi]
        for qa in para['qas']:
                qi = word_tokenize(qa['question'])
                if qa['is_impossible']:
                    labeli=[0]
                    #print(labeli)
                else:
                    labeli=[1]       
                yi = []
                yyi=[] 
                answers = []
                q_id = qa['id']
                
                ans=[]
                if labeli[0] == 1:
                    ans=qa['answers']
                else:
                    ans=qa['plausible_answers']
                    
                for answer in ans:
                    answer_text=answer['text']
                    answers.append(answer_text)
                    answer_start = answer['answer_start']
                    answer_stop = answer_start + len(answer_text)
                   # print(context)
                   # print(si)
                    yi0, yi1 = get_word_span(context, si, answer_start, answer_stop)
                    #w0 = xi[yi0[0]][yi0[1]]            
                    #w1 = xi[yi1[0]][yi1[1]-1]
                    yi.append([ai,pi,yi0[0],qi,labeli[0],q_id]) 
                    #if labeli[0] == 0:
                        #print(yi)
                   # print(yi)
                   # print(yi)
                    yyi.append([answer_start,answer_stop])
                for qij in qi:            
                         word_counter[qij] += 1
                        
                        
                        
                question_list.append(qi)
                answer_list.append(answers)
                answer_index.append(yi)
                label.append(labeli)
                label_index.append(indexi)
               # ids.append(qa['id'])
                idxs.append(len(idxs))
         
    sentence_list.append(s)

    context_list.append(p)

100%|██████████| 442/442 [00:46<00:00, 11.19it/s]


In [7]:
X, s_list, q_list, a_list, q_id_list=generatedataset(sentence_list,answer_index)   

In [7]:
ctr=0
for l in answer_index:
    try:
        if l[0][4] ==0:
            ctr+=1
    except:
        continue
ctr        

34798

In [8]:
print(len(X))
print(len(s_list))
print(len(q_list))
print(len(a_list))

69596
69596
69596
69596


In [9]:
answer_index

[[[0,
   0,
   3,
   ['What',
    'was',
    'the',
    'first',
    'album',
    'Beyoncé',
    'released',
    'as',
    'a',
    'solo',
    'artist',
    '?'],
   1,
   '56d43c5f2ccc5a1400d830ab']],
 [[0,
   0,
   1,
   ['In', 'what', 'city', 'and', 'state', 'did', 'Beyonce', 'grow', 'up', '?'],
   1,
   '56bf6b0f3aeaaa14008c9601']],
 [[0,
   0,
   2,
   ['Who', 'managed', 'the', 'Destiny', "'s", 'Child', 'group', '?'],
   1,
   '56bf6b0f3aeaaa14008c9605']],
 [[0,
   0,
   1,
   ['In', 'which', 'decade', 'did', 'Beyonce', 'become', 'famous', '?'],
   1,
   '56bf6b0f3aeaaa14008c9602']],
 [[0,
   0,
   3,
   ['How',
    'many',
    'Grammy',
    'awards',
    'did',
    'Beyoncé',
    'win',
    'for',
    'her',
    'first',
    'solo',
    'album',
    '?'],
   1,
   '56d43c5f2ccc5a1400d830ad']],
 [[0,
   1,
   0,
   ['What',
    'was',
    'the',
    'name',
    'of',
    'Beyoncé',
    "'s",
    'second',
    'solo',
    'album',
    '?'],
   1,
   '56d43da72ccc5a1400d830be']],
 

# Using pre-trained InferSent

In [21]:
# Load model
from models import InferSent
model_version = 1
MODEL_PATH = "./encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))


# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model


# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'glove.840B.300d.txt' if model_version == 1 else '../dataset/fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

# Load embeddings of K most frequent words
model.build_vocab_k_words(K=1000000)

Vocab size : 1000000


Vocab size : 1000000


# Load and encode sentences

In [6]:
def get_sent_embedding(sent_list):
    embeddings = model.encode(sent_list, bsize=128, tokenize=False, verbose=True)
    return embeddings

    

In [12]:
q_embed = get_sent_embedding(q_list)
s_embed = get_sent_embedding(s_list)

Nb words kept : 899545/906138 (99.3%)
Speed : 59.5 sentences/s (cpu mode, bsize=128)
Nb words kept : 2440306/2462124 (99.1%)
Speed : 57.6 sentences/s (cpu mode, bsize=128)


In [13]:
np.save('q_embed', q_embed)
np.save('s_embed', s_embed)


In [13]:
np.save('a_list', a_list)

In [53]:
q_embed = np.load('q_embed.npy')
s_embed = np.load('s_embed.npy')

In [67]:
a_list = np.load('a_list.npy')

In [6]:
len(q_embed)

69596

In [7]:
len(s_embed)

69596

In [170]:
q_embed[0].reshape(1,-1)

array([[ 0.03582741,  0.09328488,  0.09180234, ...,  0.05242723,
        -0.03814262,  0.0522803 ]], dtype=float32)

In [284]:
a =preprocessing.normalize(q_embed, norm='l2')

0.0

# Cosine Similarity

In [8]:
def cosineimilarity(e,v):
  
    #Input:
    #e = nxd input matrix with n row-vectors of dimensionality d (n is number of dictionary_keys)
    #v = mxd input matrix with m row-vectors of dimensionality d (m is number of test samples)
    # Output:
    # Matrix D of size nxm
    # s(i,j) is the cosinesimiarlity of embed(i,:) and test(j,:)
    g=e.dot(v.T)
    b=np.expand_dims(np.linalg.norm(e,axis=1),1)
    a=np.expand_dims(np.linalg.norm(v,axis=1),1)
    s=np.divide(g,np.multiply(b,a.T))
    # ... until here
    return s




In [ ]:
sim_mat = cosineimilarity(q_embed,s_embed)

In [9]:
def get_pred_cos_sim(sim_mat, threshold=0.6):
    preds=[]
    
    sim = sim_mat.diagonal()
    
    for i in range(sim.shape[0]):
        
        if abs(sim[i])>threshold:
            preds+=[1]
        else:
            preds+=[0]
    return sim,preds

def get_acc(preds,gold_labels):
    correct=0;
    total=len(preds)
    for i in range(total):
        if preds[i]==gold_labels[i]:
            correct+=1
    return correct/total

In [ ]:
cos_sim, preds = get_pred_cos_sim(sim_mat)


In [ ]:
Counter(preds)

In [ ]:
get_acc(preds,a_list)

In [ ]:
len(q_)

# Dev Set

In [10]:
te_data_type='development'
te_source_path = "{}.json".format(te_data_type)
te_source_data = json.load(open(te_source_path, 'r'))



te_context_list=[]
te_sentence_list=[]
te_question_list=[]
te_answer_list=[]
te_answer_index=[]
te_label_index=[]
te_label=[]
te_idxs=[]
te_word_counter = Counter()

for ai, article in enumerate(tqdm(te_source_data['data'])):
    s=[]
    p = []
    for pi, para in enumerate(article['paragraphs']):
        # wordss
        context = para['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
       
        si = list(map(word_tokenize, sent_tokenize(context)))
       # print(si)
        si = [process_tokens(tokens) for tokens in si]  # process tokens
        for sentence in si:
            for word in sentence:
                 word_counter[word] += 1
 
        # given xi, add chars
        s.append(si)
        p.append(context)

        indexi = [ai, pi]
        for qa in para['qas']:
                qi = word_tokenize(qa['question'])
                
                if qa['is_impossible']:
                    labeli=[0]
                  #  print(labeli)
                else:
                    labeli=[1] 
                       
                yi = []
                yyi=[] 
                answers = []
                q_id = qa['id']
                
                ans=[]
                if labeli[0] == 1:
                    ans=qa['answers']
                else:
                    ans=qa['plausible_answers']
                    
                
                for answer in ans:
                    answer_text=answer['text']
                    answers.append(answer_text)
                    answer_start = answer['answer_start']
                    answer_stop = answer_start + len(answer_text)
                   # print(context)
                   # print(si)
                    yi0, yi1 = get_word_span(context, si, answer_start, answer_stop)
                    #w0 = xi[yi0[0]][yi0[1]]            
                    #w1 = xi[yi1[0]][yi1[1]-1]
                    yi.append([ai,pi,yi0[0],qi,labeli[0],q_id]) 
                   # print(yi)
                   # print(yi)
                    yyi.append([answer_start,answer_stop])
                for qij in qi:
                    
                            word_counter[qij] += 1
                te_question_list.append(qi)
                te_answer_list.append(answers)
                te_answer_index.append(yi)
                te_label.append(labeli)
                te_label_index.append(indexi)
               # ids.append(qa['id'])
                te_idxs.append(len(idxs))
         
    te_sentence_list.append(s)

    te_context_list.append(p)

100%|██████████| 442/442 [00:31<00:00, 12.24it/s]


In [29]:
te_X, te_s_list, te_q_list, te_a_list, te_q_id_list=generatedataset(te_sentence_list,te_answer_index)   

17400

In [25]:
q_id

'5a7e070b70df9f001a87543a'

In [16]:
print(len(te_X))
print(len(te_s_list))
print(len(te_q_list))
print(len(te_a_list))

17400
17400
17400
17400


In [13]:
Counter(te_a_list)

Counter({0: 8700, 1: 8700})

In [22]:
te_q_embed = get_sent_embedding(te_q_list)
te_s_embed = get_sent_embedding(te_s_list)


Nb words kept : 225688/227354 (99.3%)
Speed : 160.2 sentences/s (cpu mode, bsize=128)
Nb words kept : 613261/618478 (99.2%)
Speed : 57.3 sentences/s (cpu mode, bsize=128)


In [12]:
te_q_embed = np.load('te_q_embed.npy')
te_s_embed = np.load('te_s_embed.npy')

In [23]:
model

InferSent(
  (enc_lstm): LSTM(300, 2048, bidirectional=True)
)

In [15]:
te_sim_mat = cosineimilarity(te_q_embed,te_s_embed)

In [16]:
te_cos_sim, te_preds = get_pred_cos_sim(te_sim_mat)

In [17]:
get_acc(te_preds,te_a_list)

0.5233333333333333

In [31]:
pred_dict = get_prediction_dict(te_preds, te_q_id_list)

In [35]:
import csv
write_result('model1-dev.csv',pred_dict )

In [14]:
te_q_embed.shape

(17400, 4096)

In [38]:
def calculate_precision(true_positives, false_positives):
    if true_positives + false_positives == 0:
        return 0
    return true_positives / (true_positives + false_positives)


def calculate_recall(true_positives, false_negatives):
    if true_positives + false_negatives == 0:
        return 0
    return true_positives / (true_positives + false_negatives)


def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return (2 * precision * recall) / (precision + recall)


def calculate_accuracy(true_positive, true_negative, false_positive, false_negative):
    return (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)


In [39]:
def evaluate(pred, test_set):
    """

    :param preds: A dictionary mapping question IDs to:
                    -> 0 if the question does not have a plausible answer
                    -> 1 if the question does have a plausible answer

    :param test_set: A dictionary of the following form:

    test_set["data"] -> list of titles
        title["title"] -> title name
        title["paragraphs"] -> list of paragraphs
            paragraph["context"] -> data which might answer questions in this paragraph
            paragraph["qas"] -> list of questions
                question["question"] -> question content
                question["id"] -> question ID

    :return: A 3-tuple consisting of (precision, recall, f1_score)
    """
    
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    n=len(pred)
    for i in range(n):
        if pred[i] == 0 and test_set[i] == 0:
            true_negatives += 1
        elif pred[i] == 1 and test_set[i]  == 1:
            true_positives += 1
        elif pred[i]== 0 and test_set[i] == 1:
            false_positives += 1
        elif pred[i] == 1 and test_set[i] == 0:
            false_negatives += 1

    precision = calculate_precision(true_positives, false_positives)
    recall = calculate_recall(true_positives, false_negatives)
    f_1 = calculate_f1(precision, recall)
    acc=(true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
    return precision, recall, f_1,acc

In [40]:
evaluate(te_preds,te_a_list)

(0.9105747126436782, 0.5131493716802694, 0.656392410307399, 0.5233333333333333)

# Test Set

In [31]:
te_data_type='test'
te_source_path = "{}.json".format(te_data_type)
te_source_data = json.load(open(te_source_path, 'r'))

test_q=[]
test_q_id=[]
test_sen=[]

for ai, article in enumerate(tqdm(te_source_data['data'])):
    s=[]
    p = []
    for pi, para in enumerate(article['paragraphs']):
        # wordss
        context = para['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
       
        si = list(map(word_tokenize, sent_tokenize(context)))
       # print(si)
        si = [process_tokens(tokens) for tokens in si]  # process tokens
        for sentence in si:
            for word in sentence:
                 word_counter[word] += 1
 


            for qa in para['qas']:
                    qi = word_tokenize(qa['question'])
                    q_id = qa['id']

                    test_q.append(' '.join(qi))
                    test_q_id.append(q_id)
                    test_sen.append(' '.join(sentence))


               
         


100%|██████████| 35/35 [00:08<00:00,  3.78it/s]


In [32]:
len(test_q_id)

63700

In [33]:
test_q

['In what country is Normandy located ?',
 'When were the Normans in Normandy ?',
 'From which countries did the Norse originate ?',
 'Who was the Norse leader ?',
 'What century did the Normans first gain their separate identity ?',
 "Who gave their name to Normandy in the 1000 's and 1100 's",
 'What is France a region of ?',
 'Who did King Charles III swear fealty to ?',
 'When did the Frankish identity emerge ?',
 'In what country is Normandy located ?',
 'When were the Normans in Normandy ?',
 'From which countries did the Norse originate ?',
 'Who was the Norse leader ?',
 'What century did the Normans first gain their separate identity ?',
 "Who gave their name to Normandy in the 1000 's and 1100 's",
 'What is France a region of ?',
 'Who did King Charles III swear fealty to ?',
 'When did the Frankish identity emerge ?',
 'In what country is Normandy located ?',
 'When were the Normans in Normandy ?',
 'From which countries did the Norse originate ?',
 'Who was the Norse leade

In [34]:
test_q_embed = get_sent_embedding(test_q)
test_s_embed = get_sent_embedding(test_sen)


Nb words kept : 836633/843857 (99.1%)
Speed : 144.9 sentences/s (cpu mode, bsize=128)
Nb words kept : 1890676/1907098 (99.1%)
Speed : 67.8 sentences/s (cpu mode, bsize=128)


In [35]:
np.save('test_q_embed', test_q_embed)
np.save('test_s_embed', test_s_embed)
np.save('test_q_id',test_q_id)

np.save('te_q_embed', te_q_embed)
np.save('te_s_embed', te_s_embed)


In [36]:
np.save('te_a_list', te_a_list)


In [ ]:
test_q_embed = np.load('test_q_embed.npy')
test_s_embed = np.load('test_s_embed.npy')
test_q_id = np.load('test_q_id.npy')

In [ ]:
#get cosine similarity

test_sim_mat = cosineimilarity(test_q_embed,test_s_embed)


In [47]:
#get predictions

def get_test_preds(sim_mat, q_id ,threshold=0.85):
    preds={}
    
    sim = sim_mat.diagonal()
    
    
    for i in range(sim.shape[0]):
        
        #if id not in dict then create 
        
        if q_id[i] not in preds.keys():
            preds[q_id[i]] = 0
        
        if abs(sim[i])>threshold and preds[q_id[i]] == 0:
            preds[q_id[i]] = 1
        
            
    return sim,preds

In [9]:
len((test_q_id))

63700

In [11]:
test_sim_mat[0]

array([0.62112534, 0.62112534, 0.62112534, ..., 0.47072425, 0.47072428,
       0.47427484], dtype=float32)

In [48]:
test_cos_sim, test_preds = get_test_preds(test_sim_mat, test_q_id)

In [49]:
test_preds

{'56ddde6b9a695914005b9628': 1,
 '56ddde6b9a695914005b9629': 1,
 '56ddde6b9a695914005b962a': 1,
 '56ddde6b9a695914005b962b': 1,
 '56ddde6b9a695914005b962c': 1,
 '5ad39d53604f3c001a3fe8d1': 1,
 '5ad39d53604f3c001a3fe8d2': 1,
 '5ad39d53604f3c001a3fe8d3': 1,
 '5ad39d53604f3c001a3fe8d4': 1,
 '56dddf4066d3e219004dad5f': 1,
 '56dddf4066d3e219004dad60': 1,
 '56dddf4066d3e219004dad61': 1,
 '5ad3a266604f3c001a3fea27': 1,
 '5ad3a266604f3c001a3fea28': 1,
 '5ad3a266604f3c001a3fea29': 1,
 '5ad3a266604f3c001a3fea2a': 1,
 '5ad3a266604f3c001a3fea2b': 1,
 '56dde0379a695914005b9636': 0,
 '56dde0379a695914005b9637': 1,
 '5ad3ab70604f3c001a3feb89': 0,
 '5ad3ab70604f3c001a3feb8a': 1,
 '56dde0ba66d3e219004dad75': 1,
 '56dde0ba66d3e219004dad76': 1,
 '56dde0ba66d3e219004dad77': 1,
 '5ad3ad61604f3c001a3fec0d': 1,
 '5ad3ad61604f3c001a3fec0e': 1,
 '5ad3ad61604f3c001a3fec0f': 1,
 '5ad3ad61604f3c001a3fec10': 1,
 '56dde1d966d3e219004dad8d': 1,
 '5ad3ae14604f3c001a3fec39': 1,
 '5ad3ae14604f3c001a3fec3a': 1,
 '56dde2

In [14]:
import csv

In [20]:
def write_result(filename,preds):
    with open(filename, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Id','Category'])
        for k,v in preds.items():
            filewriter.writerow([k,v])

In [51]:
write_result('baseline_cos_sim_1.csv', test_preds)

In [19]:
def get_prediction_dict(preds, q_ids):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    return result

In [ ]:
def evaluate(preds, test_set):
    """

    :param preds: A dictionary mapping question IDs to:
                    -> 0 if the question does not have a plausible answer
                    -> 1 if the question does have a plausible answer

    :param test_set: A dictionary of the following form:

    test_set["data"] -> list of titles
        title["title"] -> title name
        title["paragraphs"] -> list of paragraphs
            paragraph["context"] -> data which might answer questions in this paragraph
            paragraph["qas"] -> list of questions
                question["question"] -> question content
                question["id"] -> question ID

    :return: A 3-tuple consisting of (precision, recall, f1_score)
    """

    correct_mapping = {}

    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0

    for title in test_set["data"]:
        for paragraph in title["paragraphs"]:
            for question in paragraph["qas"]:
                if question["is_impossible"]:
                    correct_mapping[question["id"]] = 0
                else:
                    correct_mapping[question["id"]] = 1

    for id, pred in correct_mapping.items():
        if pred == 0 and preds[id] == 0:
            true_negatives += 1
        elif pred == 1 and preds[id] == 1:
            true_positives += 1
        elif pred == 0 and preds[id] == 1:
            false_positives += 1
        elif pred == 1 and preds[id] == 0:
            false_negatives += 1

    print("Intermediate evaluation:")
    print("true_positives: " + str(true_positives))
    print("true_negatives: " + str(true_negatives))
    print("false_positives: " + str(false_positives))
    print("false_negatives: " + str(false_negatives))
    print("\n" * 3)

    precision = calculate_precision(true_positives, false_positives)
    recall = calculate_recall(true_positives, false_negatives)
    f_1 = calculate_f1(precision, recall)
    accuracy = calculate_accuracy(true_positives, true_negatives, false_positives, false_negatives)
    return precision, recall, f_1, accuracy


def calculate_precision(true_positives, false_positives):
    if true_positives + false_positives == 0:
        return 0
    return true_positives / (true_positives + false_positives)


def calculate_recall(true_positives, false_negatives):
    if true_positives + false_negatives == 0:
        return 0
    return true_positives / (true_positives + false_negatives)


def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return (2 * precision * recall) / (precision + recall)


def calculate_accuracy(true_positive, true_negative, false_positive, false_negative):
    return (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)

